In [19]:
import tensorflow as tf
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import load_model
from utils.real_time_tools import RealTimeTools

In [20]:
model = load_model('../../model/2025-09-16/cnn_1d_model.keras', safe_mode=False)

In [15]:
from tensorflow.keras.models import load_model
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Lambda

# Funkcje użyte w Lambda
def softmax_lambda(z):
    return K.softmax(z, axis=1)

def sum_lambda(z):
    return K.sum(z, axis=1)

model = load_model(
    '../../model/2025-09-16/bilstm_attention_model.keras',
    custom_objects={
        'softmax_lambda': softmax_lambda,
        'sum_lambda': sum_lambda
    },
    safe_mode=False
)


In [21]:
with open('../../model/2025-07-14/labels.txt', 'r') as f:
    label_names = [line.strip() for line in f.readlines()]

In [22]:
import cv2
import mediapipe as mp

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.7
)

W0000 00:00:1758058895.719630  281235 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1758058895.742534  281235 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [23]:
cap = cv2.VideoCapture(0)
try:
    if not cap.isOpened():
        print("Camera is not opened")
        exit()

    while True:
        ret, frame = cap.read()
        if not ret:
            print("No frames")
            break

        image_rbg = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(image_rbg)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                landmarks = RealTimeTools.extract_hand_landmarks(hand_landmarks)

        cv2.imshow("Gesture Recognition", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

except KeyboardInterrupt:
    print("Camera stopped")
finally:
    cap.release()
    cv2.destroyAllWindows()

